In [1]:
import os

In [2]:
%pwd

'd:\\model deploy end to end\\NLP-text_classification\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\model deploy end to end\\NLP-text_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    clean_data_path: Path

In [6]:
from src.NLPtc.constants import *
from src.NLPtc.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            clean_data_path=config.clean_data_path
        )

        return data_transformation_config

In [8]:
import os
from src.NLPtc import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def data_transform(self):
        data= pd.read_csv(self.config.clean_data_path,sep='\t',names=['text','label'])
        
        cor = data['text']
        tv = TfidfVectorizer(max_features=2500)
        x = tv.fit_transform(cor).toarray()
        x = pd.DataFrame(x)
        y = pd.get_dummies(data['label'],dtype=int,drop_first=True)
        
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)
        
        X_train.to_csv(os.path.join(self.config.root_dir, "xtrain.csv"),index = False)
        X_test.to_csv(os.path.join(self.config.root_dir, "xtest.csv"),index = False)
        y_train.to_csv(os.path.join(self.config.root_dir, "ytrain.csv"),index = False)
        y_test.to_csv(os.path.join(self.config.root_dir, "ytest.csv"),index = False)
        
        logger.info(X_train.shape)
        logger.info(X_test.shape)
        logger.info(y_train.shape)
        logger.info(y_test.shape)

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.data_transform()

except Exception as e:
    raise e

[2023-10-03 22:41:22,394: INFO: common: yaml file: D:\model deploy end to end\NLP-text_classification\config\config.yaml loaded successfully]
[2023-10-03 22:41:22,397: INFO: common: yaml file: D:\model deploy end to end\NLP-text_classification\params.yaml loaded successfully]
[2023-10-03 22:41:22,403: INFO: common: yaml file: D:\model deploy end to end\NLP-text_classification\schema.yaml loaded successfully]
[2023-10-03 22:41:22,408: INFO: common: created directory at: artifacts]
[2023-10-03 22:41:22,410: INFO: common: created directory at: artifacts/data_transformation]
[2023-10-03 22:41:29,036: INFO: 1164633394: (4455, 2500)]
[2023-10-03 22:41:29,036: INFO: 1164633394: (1114, 2500)]
[2023-10-03 22:41:29,036: INFO: 1164633394: (4455, 0)]
[2023-10-03 22:41:29,036: INFO: 1164633394: (1114, 0)]
